In [ ]:
!pip install --upgrade pandas_profiling 
# cần update để pandas_profiling tương thích với các packages khác

In [17]:
# View and understanding data
# data exploration
# %%timeit 
# calculate running time to check ways for accelerating the processing of pandas: dask.dataframe is fastest way betwween chunksize,modin,dask
# test vaex
import pandas as pd
import dask.dataframe as dd
train_full=pd.read_csv('/content/drive/My Drive/Google colab data/Elo_merchant_kaggle/train.csv')
merchants=pd.read_csv('/content/drive/My Drive/Google colab data/Elo_merchant_kaggle/merchants.csv')
test=pd.read_csv('/content/drive/My Drive/Google colab data/Elo_merchant_kaggle/test.csv')

In [ ]:
table_list=[train_full,new_trans,merchants,test]
for i in table_list:
  print (i.info())
  print (i.describe())
new_trans.head()
# test vs new_trains.compute() vs new_trains

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201917 entries, 0 to 201916
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   first_active_month  201917 non-null  object 
 1   card_id             201917 non-null  object 
 2   feature_1           201917 non-null  int64  
 3   feature_2           201917 non-null  int64  
 4   feature_3           201917 non-null  int64  
 5   target              201917 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 9.2+ MB
None
           feature_1      feature_2      feature_3         target
count  201917.000000  201917.000000  201917.000000  201917.000000
mean        3.105311       1.745410       0.565569      -0.393636
std         1.186160       0.751362       0.495683       3.850500
min         1.000000       1.000000       0.000000     -33.219281
25%         2.000000       1.000000       0.000000      -0.883110
50%         3.000000       

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [ ]:
import pandas_profiling 
new_trans_report=pandas_profiling.ProfileReport(new_trans)
new_trans_report
# data visualization

In [ ]:
print(new_trans.info())
new_trans.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1963031 entries, 0 to 1963030
Data columns (total 14 columns):
 #   Column                Dtype  
---  ------                -----  
 0   authorized_flag       object 
 1   card_id               object 
 2   city_id               int64  
 3   category_1            object 
 4   installments          int64  
 5   category_3            object 
 6   merchant_category_id  int64  
 7   merchant_id           object 
 8   month_lag             int64  
 9   purchase_amount       float64
 10  purchase_date         object 
 11  category_2            float64
 12  state_id              int64  
 13  subsector_id          int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 209.7+ MB
None


authorized_flag              0
card_id                      0
city_id                      0
category_1                   0
installments                 0
category_3               55922
merchant_category_id         0
merchant_id              26216
month_lag                    0
purchase_amount              0
purchase_date                0
category_2              111745
state_id                     0
subsector_id                 0
dtype: int64

In [39]:
# fEATURE enginerring
# Test data
test['first_active_month']=pd.to_datetime(train_full['first_active_month'])
test['elapsed_time']=(datetime.date(2018,2,1)-train_full['first_active_month'].dt.date).dt.days
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time
0,2017-06-01,C_ID_0ab67a22ab,3,3,1,245
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,396
2,2016-08-01,C_ID_b709037bc5,5,1,1,549
3,2017-09-01,C_ID_d27d835a9f,2,1,0,153
4,2017-11-01,C_ID_2b5e3df5c2,5,1,1,92


In [18]:
# Train_data

train_full['first_active_month']=pd.to_datetime(train_full['first_active_month'])
train_full['elapsed_time']=(datetime.date(2018,2,1)-train_full['first_active_month'].dt.date).dt.days #df['firstactive_month'].max() = (2018,2,1)

target=train_full['target']
del train_full['target']
train_full.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time
0,2017-06-01,C_ID_92a2005557,5,2,1,245
1,2017-01-01,C_ID_3d0044924f,4,1,0,396
2,2016-08-01,C_ID_d639edf6cd,2,2,0,549
3,2017-09-01,C_ID_186d6a6901,4,3,0,153
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,92


In [10]:

from ipykernel import kernelapp as app
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def binarize(df):
    for i in ['authorized_flag','category_1']:
        df[i]=df[i].map({'Y':1, 'N':0})
    return df
#create function to aggregate info
def aggregation_his(history):
    history['purchase_date']=pd.to_datetime(history['purchase_date']).astype(np.int64)
    agg_func={
    'category_1': ['sum', 'mean'],
    'category_2_1.0': ['mean'],
    'category_2_2.0': ['mean'],
    'category_2_3.0': ['mean'],
    'category_2_4.0': ['mean'],
    'category_2_5.0': ['mean'],
    'category_3_A': ['mean'],
    'category_3_B': ['mean'],
    'category_3_C': ['mean'],
    'merchant_id': ['nunique'],
    'merchant_category_id': ['nunique'],
    'state_id': ['nunique'],
    'city_id': ['nunique'],
    'subsector_id': ['nunique'],
    'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
    'installments': ['sum', 'mean', 'max', 'min', 'std'],
    'purchase_month': ['mean', 'max', 'min', 'std'],
    'purchase_date': ['min', 'max'],
    'month_lag': ['mean', 'max', 'min', 'std'],
    'month_diff': ['mean']
    }
    agg_his=history.groupby('card_id').agg(agg_func)
    # Transform multiindex to index
    agg_his.columns=['_'.join(i) for i in agg_his.columns]
    agg_his.reset_index(inplace=True)

    df= history.groupby('card_id').size().reset_index(name='transactions_count')
    agg_his_trans=pd.merge(agg_his,df,on='card_id',how='left')
    return agg_his_trans
 
def aggregate_per_month(history):
    grouped=history.groupby(['card_id','month_lag'])
    agg_fuc={
        'purchase_amount':['count','sum','mean','min','max','std'],
        'installments':['count','sum','mean','min','max','std']
    }
    intermediate_group=grouped.agg(agg_fuc)
    intermediate_group.columns=['_'.join(i) for i in intermediate_group.columns]
    intermediate_group.reset_index(inplace=True)

    final_group=intermediate_group.groupby('card_id').agg(['mean','std'])
    final_group.columns=['_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True)
    return final_group  

In [12]:
# feature engineering with historical transaction data
his_trans=pd.read_csv('/content/drive/My Drive/Google colab data/Elo_merchant_kaggle/historical_transactions.csv',chunksize=100000)

his_list=[]   
authorized_his_list=[]
final_group_list=[]
for chunk in his_trans:
    chunk=binarize(chunk)

    chunk=pd.get_dummies(chunk,columns=['category_2','category_3'])
    #chunk=reduce_mem_usage(chunk)

    auth_mean=chunk.groupby('card_id').agg({'authorized_flag':'mean'})
    auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
    auth_mean.reset_index(inplace=True)
    
    # Tổng hợp theo authorized_flag
    authorized_his_trans=chunk[chunk['authorized_flag']==1]
    his_trans=chunk[chunk['authorized_flag']==0]

    his_trans['purchase_month']=pd.to_datetime(his_trans['purchase_date']).dt.month
    authorized_his_trans['purchase_month']=pd.to_datetime(authorized_his_trans['purchase_date']).dt.month

    his=aggregation_his(his_trans)
    his.columns=['hist_'+ c if c !='card_id' else c for c in his.columns]
    authorized_his=aggregation_his(authorized_his_trans)
    authorized_his.columns=['author_'+ c if c !='card_id' else c for c in authorized_his.columns]

    his_list.append(his)
    authorized_his_list.append(authorized_his)

    final_group=aggregate_per_month(authorized_his_trans)
    final_group_list.append(final_group)

no_authorized_his=pd.concat(his_list)
authorized_his=pd.concat(authorized_his_list)
final_group=pd.concat(final_group_list)

no_authorized_his.head()

,card_id,hist_category_1_sum,hist_category_1_mean,hist_category_2_1.0_mean,hist_category_2_2.0_mean,hist_category_2_3.0_mean,hist_category_2_4.0_mean,hist_category_2_5.0_mean,hist_category_3_A_mean,hist_category_3_B_mean,hist_category_3_C_mean,hist_merchant_id_nunique,hist_merchant_category_id_nunique,hist_state_id_nunique,hist_city_id_nunique,hist_subsector_id_nunique,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_mean,hist_installments_max,hist_installments_min,hist_installments_std,hist_purchase_month_mean,hist_purchase_month_max,hist_purchase_month_min,hist_purchase_month_std,hist_purchase_date_min,hist_purchase_date_max,hist_month_lag_mean,hist_month_lag_max,hist_month_lag_min,hist_month_lag_std,hist_month_diff_mean,hist_transactions_count
0,C_ID_0046db9f8a,0,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.250000,0.500000,5,5,2,2,4,1.175419,0.146927,3.625281,-0.704834,1.423650,15,1.875000,6,-1,2.356602,6.250000,12,1,5.625198,1511395200000000000,1515677076000000000,-1.750000,-1,-3,0.886405,29.000000,8
1,C_ID_01904d743d,0,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.952381,0.047619,0.000000,6,6,1,1,5,-14.954189,-0.712104,-0.545583,-0.740011,0.042330,1,0.047619,1,0,0.218218,8.523810,12,2,3.280534,1500660263000000000,1518249224000000000,-3.761905,0,-7,2.406341,28.904762,21
2,C_ID_01b098ff01,16,0.296296,0.703704,0.0,0.0,0.0,0.000000,0.000000,0.759259,0.240741,19,11,3,10,9,-31.260178,-0.578892,0.004418,-0.745405,0.228692,109,2.018519,6,1,1.937619,5.870370,12,1,3.022221,1489181477000000000,1515773126000000000,-6.018519,0,-10,3.230110,29.981481,54
3,C_ID_02c2ed9fc0,2,0.153846,0.076923,0.0,0.0,0.0,0.769231,0.000000,0.923077,0.076923,8,7,3,4,5,-4.939004,-0.379923,2.555198,-0.725871,0.895349,14,1.076923,2,1,0.277350,9.153846,12,7,2.267383,1500554084000000000,1513725690000000000,-4.846154,-2,-7,2.267383,28.923077,13
4,C_ID_02cc9ba024,0,0.000000,0.000000,0.0,0.0,0.0,1.000000,0.916667,0.083333,0.000000,21,9,2,3,7,-24.025514,-0.667375,-0.386572,-0.745405,0.073056,3,0.083333,1,0,0.280306,7.416667,12,1,3.612478,1489836148000000000,1519048701000000000,-4.250000,0,-11,2.862317,28.861111,36


In [23]:
# new merchant transactions
new_trans=pd.read_csv('/content/drive/My Drive/Google colab data/Elo_merchant_kaggle/new_merchant_transactions.csv')
new_trans = binarize(new_trans)
new_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,C_ID_415bb3a509,107,0,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,1,C_ID_415bb3a509,140,0,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,1,C_ID_415bb3a509,330,0,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,1,C_ID_415bb3a509,-1,1,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,1,C_ID_ef55cf8d4b,-1,1,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [25]:
values={'category_2':1.0,'category_3':'A'}
new_trans.fillna(value=values,inplace=True)
new_trans=pd.get_dummies(new_trans,columns=['category_2','category_3'])

In [24]:
auth_mean=new_trans.groupby('card_id').agg({'authorized_flag':'mean'})
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
auth_mean.reset_index(inplace=True)

In [41]:
new_trans['month_diff']=(datetime.datetime.today()-pd.to_datetime(new_trans['purchase_date'])).dt.days//30
new_trans['month_diff']+=new_trans['month_lag']

In [43]:
new_trans['purchase_month']=pd.to_datetime(new_trans['purchase_date']).dt.month 
new=aggregation_his(new_trans)
new.head()

,card_id,category_1_sum,category_1_mean,category_2_1.0_mean,category_2_2.0_mean,category_2_3.0_mean,category_2_4.0_mean,category_2_5.0_mean,category_3_A_mean,category_3_B_mean,category_3_C_mean,merchant_id_nunique,merchant_category_id_nunique,state_id_nunique,city_id_nunique,subsector_id_nunique,purchase_amount_sum,purchase_amount_mean,purchase_amount_max,purchase_amount_min,purchase_amount_std,installments_sum,installments_mean,installments_max,installments_min,installments_std,purchase_month_mean,purchase_month_max,purchase_month_min,purchase_month_std,purchase_date_min,purchase_date_max,month_lag_mean,month_lag_max,month_lag_min,month_lag_std,month_diff_mean,transactions_count
0,C_ID_00007093c1,0,0.000000,0.500000,0.000000,0.50000,0.0,0.000000,0.000000,1.000000,0.000000,2,2,2,2,2,-1.328524,-0.664262,-0.656749,-0.671775,0.010625,2,1.000000,1,1,0.000000,4.000000,4,4,0.000000,1522754015000000000,1523291039000000000,2.000000,2,2,0.000000,29.000000,2
1,C_ID_0001238066,2,0.076923,0.884615,0.000000,0.00000,0.0,0.115385,0.038462,0.807692,0.153846,25,15,4,8,9,-14.850055,-0.571156,-0.078318,-0.740897,0.173436,42,1.615385,10,-1,2.079941,3.346154,4,3,0.485165,1519922907000000000,1525118250000000000,1.346154,2,1,0.485165,28.807692,26
2,C_ID_0001506ef0,0,0.000000,0.000000,0.000000,1.00000,0.0,0.000000,1.000000,0.000000,0.000000,1,2,1,1,2,-1.447354,-0.723677,-0.715352,-0.732001,0.011773,0,0.000000,0,0,0.000000,3.000000,3,3,0.000000,1521238918000000000,1521710070000000000,1.000000,1,1,0.000000,29.000000,2
3,C_ID_0001793786,0,0.000000,0.548387,0.258065,0.16129,0.0,0.032258,1.000000,0.000000,0.000000,31,21,5,7,14,-0.229620,-0.007407,3.129932,-0.737892,0.947223,0,0.000000,0,0,0.000000,11.322581,12,11,0.475191,1510760660000000000,1514741756000000000,1.322581,2,1,0.475191,32.709677,31
4,C_ID_000183fdda,0,0.000000,0.000000,0.000000,1.00000,0.0,0.000000,0.090909,0.545455,0.363636,11,9,2,2,6,-6.590778,-0.599162,-0.107680,-0.732332,0.182877,16,1.454545,4,-1,1.293340,3.272727,4,3,0.467099,1519993586000000000,1525100393000000000,1.272727,2,1,0.467099,28.818182,11


In [29]:
# Hàm này để tổng hợp purchase_amount và installments theo các cách nhóm khác nhau
def successive_agg(df, field1, field2):
    u=df.groupby(['card_id',field1])[field2].mean().reset_index().groupby('card_id')[field2].agg(['mean','min','max','std'])
    #u=pd.DataFrame(t).reset_index().groupby('card_id').[field2].agg(['mean','min','max','std'])
    u.columns=[field1+'_'+field2+'_'+col for col in u.columns.values]
    u.reset_index(inplace=True)
    return u
additional_fields=successive_agg(new_trans,'category_1','purchase_amount')
additional_fields=additional_fields.merge(successive_agg(new_trans,'installments','purchase_amount'),on='card_id',how='left')
additional_fields=additional_fields.merge(successive_agg(new_trans,'category_1','installments'),on='card_id',how='left')
additional_fields.head()

,card_id,category_1_purchase_amount_mean,category_1_purchase_amount_min,category_1_purchase_amount_max,category_1_purchase_amount_std,installments_purchase_amount_mean,installments_purchase_amount_min,installments_purchase_amount_max,installments_purchase_amount_std,category_1_installments_mean,category_1_installments_min,category_1_installments_max,category_1_installments_std
0,C_ID_00007093c1,-0.664262,-0.664262,-0.664262,NaN,-0.664262,-0.664262,-0.664262,NaN,1.000000,1.000000,1.000000,NaN
1,C_ID_0001238066,-0.607445,-0.650332,-0.564558,0.060652,-0.423876,-0.625781,-0.152008,0.222019,2.479167,1.458333,3.500000,1.443676
2,C_ID_0001506ef0,-0.723677,-0.723677,-0.723677,NaN,-0.723677,-0.723677,-0.723677,NaN,0.000000,0.000000,0.000000,NaN
3,C_ID_0001793786,-0.007407,-0.007407,-0.007407,NaN,-0.007407,-0.007407,-0.007407,NaN,0.000000,0.000000,0.000000,NaN
4,C_ID_000183fdda,-0.599162,-0.599162,-0.599162,NaN,-0.491234,-0.696173,-0.107680,0.233882,1.454545,1.454545,1.454545,NaN


In [45]:
# merge all the dataframe
train=pd.merge(train_full,no_authorized_his, on='card_id', how='left')
test=pd.merge(test,no_authorized_his, on='card_id', how='left')

train=pd.merge(train,authorized_his, on='card_id', how='left')
test=pd.merge(test,authorized_his, on='card_id', how='left')

train=pd.merge(train,new, on='card_id', how='left')
test=pd.merge(test,new, on='card_id', how='left')

train=pd.merge(train,final_group, on='card_id', how='left')
test=pd.merge(test,final_group, on='card_id', how='left')

train=pd.merge(train,auth_mean, on='card_id', how='left')
test=pd.merge(test,auth_mean, on='card_id', how='left')

train=pd.merge(train,additional_fields, on='card_id', how='left')
test=pd.merge(test,additional_fields, on='card_id', how='left')

In [46]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 202999 entries, 0 to 202998
Columns: 156 entries, first_active_month to category_1_installments_std
dtypes: datetime64[ns](1), float64(134), int64(20), object(1)
memory usage: 243.2+ MB


In [47]:
pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=ff2be66cf54ea66802a19257e8549661eed625c91216d6ef5215bc52a6522089
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [48]:
features=[c for c in train.columns if c not in ['card_id','first_active_month']]
import numpy as np
import sklearn
from xgboost import XGBClassifier
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

In [54]:
target.describe()

count    201917.000000
mean         -0.393636
std           3.850500
min         -33.219281
25%          -0.883110
50%          -0.023437
75%           0.765453
max          17.965068
Name: target, dtype: float64

In [55]:
train_xgb = xgb.DMatrix(train[features].values, label= target.values)

def bayes_xgboost(learning_rate,
                  n_estimators,
                  gamma,
                  subsample,
                  colsample,
                  max_depth,
                  min_child_weight):
  #max_depth = int(max_depth)
  #n_estimators = int(n_estimators)

  param = {'learning_rate':learning_rate, 'n_estimators':int(n_estimators),'gamma': gamma,
           'subsample': subsample, 'colsample': colsample, 'max_depth':int(max_depth),
           'min_child_weight': min_child_weight}
  estimator = xgb.cv(
        param,
        dtrain =train_xgb,
        nfold = 5,
        stratified = False,
        num_boost_round = 100,
        metrics='rmse',
        )
  return -1.0 * estimator['test-rmse-mean'].iloc[-1] # Nhan với -1 vì bayes optimization chỉ tìm max của function f( estimator)

pbounds = {
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 1000),
    'max_depth': (3,10)
    'subsample': (0.1, 1.0),  # Change for big datasets
    'colsample': (0.1, 1.0),  # Change for datasets with lots of features
    'gamma': (0, 5),
    'min_child_weight':(0,20)}

optimizer = BayesianOptimization(
    f=bayes_xgboost,
    pbounds=pbounds,
    verbose=2,
    random_state=1,

)

optimizer.maximize(init_points=2,n_iter=3)
print(optimizer.max)

|   iter    |  target   | colsample |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        | -3.854    |  0.4753   |  3.602    |  0.01011  |  5.116    |  2.935    |  183.1    |  0.2676   |
|  2        | -3.927    |  0.411    |  1.984    |  0.5434   |  5.934    |  13.7     |  284.0    |  0.8903   |
|  3        | -3.854    |  0.1      |  5.0      |  0.01     |  3.0      |  0.0      |  1e+03    |  0.1      |
|  4        | -1.76e+16 |  1.0      |  0.0      |  1.0      |  10.0     |  20.0     |  100.0    |  1.0      |
|  5        | -1.47e+16 |  0.1      |  5.0      |  0.01     |  3.0      |  0.0      |  1e+03    |  0.1      |
{'target': -3.854132, 'params': {'colsample': 0.47531980423231657, 'gamma': 3.6016224672107904, 'learning_rate': 0.010113231069171438, 'max_depth': 5.116328008422879, 'min_child_weight': 2.935117816342261, 'n_estimators': 183.1047